In [1]:
# Import the necessary libraries
from pyspark.sql import SparkSession
# Create a SparkSession object
spark = SparkSession.builder.getOrCreate()

## Write code to read "churn.csv" that is available in your path in a variable called 'spark_df'. Make sure you include options 'header=T' and 'inferSchema=True'
telco_df = spark.read.csv("churn.csv", header=True, inferSchema=True)

## Write code to look at the first five rows of the "churn_df" DataFrame
telco_df.show(5)
# Import the necessary libraries
from pyspark.sql import SparkSession
# Create a SparkSession object
spark = SparkSession.builder.getOrCreate()

## Write code to read "churn.csv" that is available in your path in a variable called 'spark_df'. Make sure you include options 'header=T' and 'inferSchema=True'
telco_df = spark.read.csv("churn.csv", header=True, inferSchema=True)

## Write code to look at the first five rows of the "churn_df" DataFrame
telco_df.show(5)

## Write code to check the schema of the "churn_df" DataFrame

telco_df.printSchema()
## Write code to check the schema of the "churn_df" DataFrame

telco_df.printSchema()

24/10/01 18:41:37 WARN Utils: Your hostname, LAP-0285 resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlp0s20f3)
24/10/01 18:41:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/01 18:41:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/01 18:41:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/01 18:41:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/10/01 18:41:43 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-----+-------------+----------+-----------+--------+--------------------+---------+-----------+------+--------------+-------+----------+-------------+-------------+----------------+----------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+-----------------+----------------+---------------+-------------+-----------+-----------+-----------+----+--------------------+
|CustomerID|Count|      Country|     State|       City|Zip Code|            Lat Long| Latitude|  Longitude|Gender|Senior Citizen|Partner|Dependents|Tenure Months|Phone Service|  Multiple Lines|Internet Service|    Online Security|      Online Backup|  Device Protection|       Tech Support|       Streaming TV|   Streaming Movies|      Contract|Paperless Billing|  Payment Method|Monthly Charges|Total Charges|Churn Label|Churn Value|Churn Score|CLTV|        Churn Reason|
+----------+-----+-------------+----------+-----------

## Task 2: Data preprocessing and transformation
In this task, we’ll preprocess and transform the data to gain insights into potential predictors of customer churn. These operations will help prepare the data for further analysis or modeling. By preprocessing and transforming the data, we’ll gain insights into potential predictors of churn and prepare the data for further analysis and modeling.

##  2.1 Filter churned customers and count the number of customers
Filter the data to select only the customers who have churned and count the number of such customers. By doing so, we can focus specifically on this group and study their characteristics.

##  2.2 Group data by sex and compute average monthly charges
Group the data by sex and calculate the average monthly charges for each sex. This analysis will help us determine if there are any sex-based differences in charges and their potential impact on churn.

##  2.3 Compute total charges for each customer
Calculate the total charges for each customer by multiplying the monthly charges by the tenure and add a new column, “Total Charges,” to the DataFrame. This information will provide insights into the revenue generated from each customer and its relationship with churn.

##  2.4 Compute the correlation between monthly charges and total charges
Compute the correlation between monthly charges and total charges to determine if there is a strong relationship and how it may impact customer churn.

In [2]:

# Import the necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, corr

# Create a SparkSession object
spark = SparkSession.builder.getOrCreate()

## Write code to read "churn.csv" in a variable called 'telco_df'. Make sure you include options 'header=T' and 'inferSchema=True'
telco_df = spark.read.csv("churn.csv", header=True, inferSchema=True)

## Filter the data to select only the customers who have churned and count the number of such customers.
churn_count = telco_df.filter(col("Churn Value") == 1).count()
print(f'Counting the number of churned customers: {churn_count}')

## Group the data by gender and compute the average monthly charges for each gender
telco_df.groupBy("Gender").avg("Monthly Charges").show()

## Multiply monthly charges and tenure months columns to get the new column "Total Charges" and print the new column along with the original columns
telco_df.withColumn("Total Charges", col("Monthly Charges") * col("Tenure Months")).show()

## Compute and print the correlation between the monthly charges and the total charges
telco_df.select(corr(col("Monthly Charges"), col("Total Charges"))).show()

Counting the number of churned customers: 444
+------+--------------------+
|Gender|avg(Monthly Charges)|
+------+--------------------+
|Female|   66.05086705202314|
|  Male|   62.90597949886115|
+------+--------------------+

+----------+-----+-------------+----------+---------------+--------+--------------------+---------+-----------+------+--------------+-------+----------+-------------+-------------+----------------+----------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+-----------------+--------------------+---------------+------------------+-----------+-----------+-----------+----+--------------------+
|CustomerID|Count|      Country|     State|           City|Zip Code|            Lat Long| Latitude|  Longitude|Gender|Senior Citizen|Partner|Dependents|Tenure Months|Phone Service|  Multiple Lines|Internet Service|    Online Security|      Online Backup|  Device Protection|       Tech S

## Task 3: Exploratory data analysis (EDA)
EDA is the process of analyzing and understanding datasets to summarize their main characteristics and uncover patterns, relationships, and trends. In this task, we’ll focus on exploring the relationship between customer churn and various factors, such as contract type, tenure, partner/dependents, payment method, and state.

## 3.1 Relationship between customer churn and contract type
Compute the churn rate for each contract type (month-to-month, one-year, and two-year) and visualize the results using a bar chart. This analysis will help us understand how different contract types impact customer churn.

## 3.2 Relationship between customer churn and tenure
Explore the relationship between customer churn and tenure, which refers to the duration of the customer’s relationship with the company. Calculate the average tenure for churned and non-churned customers and compare them. This analysis will provide insights into whether customer loyalty, as indicated by tenure, affects churn.

## 3.3 Relationship between customer churn and payment method
Examine the relationship between customer churn and the payment method chosen by customers. Calculate the churn rate for each payment method to understand its impact on churn. This analysis will help us determine if certain payment methods are associated with higher or lower churn rates.

In [4]:
# Import the necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, count, avg

# Create a SparkSession object
spark = SparkSession.builder.getOrCreate()

## Write code to read "churn.csv" in a variable called 'telco_df'. Make sure you include options 'header=T' and 'inferSchema=True'
telco_df = spark.read.csv("churn.csv", header=True, inferSchema=True)

## Relationship between customer churn and contract type
telco_df.groupBy("Contract").agg((sum("Churn Value") / telco_df.count()).alias("Aggregated Churn Value")).show()

## Relationship between customer churn and tenure
telco_df.groupBy("Churn Value").agg(avg("Tenure Months").alias("Aggregated Churn Value")).show()

## Relationship between customer churn and payment method
telco_df.groupBy("Payment Method").agg((sum("Churn Value") / telco_df.count()).alias("Aggregated Churn Value")).show()

+--------------+----------------------+
|      Contract|Aggregated Churn Value|
+--------------+----------------------+
|Month-to-month|   0.22547332185886404|
|      One year|  0.024670109007458405|
|      Two year|  0.004589787722317842|
+--------------+----------------------+

+-----------+----------------------+
|Churn Value|Aggregated Churn Value|
+-----------+----------------------+
|          1|     18.09234234234234|
|          0|     38.01539645881447|
+-----------+----------------------+

+--------------------+----------------------+
|      Payment Method|Aggregated Churn Value|
+--------------------+----------------------+
|Credit card (auto...|   0.02868617326448652|
|        Mailed check|   0.04647160068846816|
|Bank transfer (au...|   0.03270223752151463|
|    Electronic check|   0.14687320711417096|
+--------------------+----------------------+



In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, corr, avg
spark = SparkSession.builder.getOrCreate()

# Task1: Loading Customer Data into a PySpark DataFrame
print("Reading 'churn.csv' into spark_df:")
telco_df = spark.read.csv("./churn.csv", header=True, inferSchema=True)
print("First 5 rows of the telco_df:")
telco_df.show(2, truncate=False, vertical=True)
print("Schema of the telco_df:")
telco_df.printSchema()

# Task 2: Preprocessing and Transformation of Data
churn_count = telco_df.filter(col("Churn Value") == 1).count()
print("Counting the number of churned customers:",churn_count)
print("Computing the average monthly charges by gender:")
telco_df.groupBy("Gender").avg("Monthly Charges").show()
print("Creating a new column 'Total Charges' by multiplying Monthly Charges and Tenure Months:")
telco_df.withColumn("Total Charges", col("Monthly Charges") * col("Tenure Months")).show(2, vertical=True)
print("Computing the correlation between Monthly Charges and Total Charges:")
telco_df.select(corr(col("Monthly Charges"), col("Total Charges"))).show()

# Task 3: EDA
print("Calculating the churn rates by contract type:")
telco_df.groupBy("Contract").agg((sum("Churn Value") / telco_df.count()).alias("Aggregated Churn Value")).show()
print("Calculating the average tenure by churn value:")
telco_df.groupBy("Churn Value").agg(avg("Tenure Months").alias("Aggregated Churn Value")).show()
print("Calculating the churn rates by payment method:")
telco_df.groupBy("Payment Method").agg((sum("Churn Value") / telco_df.count()).alias("Aggregated Churn Value")).show()

Reading 'churn.csv' into spark_df:
First 5 rows of the telco_df:
-RECORD 0-----------------------------------------
 CustomerID        | 3668-QPYBK                   
 Count             | 1                            
 Country           | United States                
 State             | California                   
 City              | Los Angeles                  
 Zip Code          | 90003                        
 Lat Long          | 33.964131, -118.272783       
 Latitude          | 33.964131                    
 Longitude         | -118.272783                  
 Gender            | Male                         
 Senior Citizen    | No                           
 Partner           | No                           
 Dependents        | No                           
 Tenure Months     | 2                            
 Phone Service     | Yes                          
 Multiple Lines    | No                           
 Internet Service  | DSL                          
 Online Security 